In [1]:
import math

import sys
sys.path.insert(0, '..')
#import d2l
import torch
import torch.nn as nn
import torch.optim as optim
import time
import numpy
import yacht_main as yacht

In [2]:
INPUT_SIZE = 6
OUTPUT_SIZE = 5

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(INPUT_SIZE)
    
net = torch.nn.Sequential(
    Reshape(),
    
    nn.Linear(in_features=INPUT_SIZE, out_features=50),
    nn.LeakyReLU(),
    nn.Linear(50,30),
    nn.LeakyReLU(),
    nn.Linear(30,OUTPUT_SIZE)
    
    #nn.Linear(in_features=INPUT_SIZE, out_features = OUTPUT_SIZE)
)

def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.uniform_(m.weight,-0.5,0.5)

net = net.float()
net.apply(init_weights)

Sequential(
  (0): Reshape()
  (1): Linear(in_features=6, out_features=50, bias=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=50, out_features=30, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Linear(in_features=30, out_features=5, bias=True)
)

In [3]:
def try_gpu():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

device = try_gpu()
print(torch.cuda.is_available())
net = net.to(device)

True


In [4]:
train_data = []
SCALE = 30.0


def pack(org):
    return (org/SCALE)

def unpack(norm):
    return round(SCALE*float(norm))

for d1 in range(6):
    for d2 in range(6):
        for d3 in range(6):
            for d4 in range(6):
                for d5 in range(6):
                    dices = [d1+1,d2+1,d3+1,d4+1,d5+1]
                    scores = [pack(f(dices)) for f in yacht.score_func[7:]]
                    #scores = [yacht.score_func[6](dices)/SCALE]
                    
                    dt = torch.tensor([dices.count(i+1) for i in range(6)], dtype = torch.float32, requires_grad = False)
                    st = torch.tensor(scores, dtype = torch.float32, requires_grad = False)
                    
                    train_data.append((dt,st))

print(str(len(train_data)))

for x,y in train_data:
    cont = ""
    for a in x:
        cont += str(int(a))+" "
    cont += " | "
    for b in y:
        cont += str(unpack(b))+" "
    print(cont)


7776
5 0 0 0 0 0  | 5 5 0 0 50 
4 1 0 0 0 0  | 6 0 0 0 0 
4 0 1 0 0 0  | 7 0 0 0 0 
4 0 0 1 0 0  | 8 0 0 0 0 
4 0 0 0 1 0  | 9 0 0 0 0 
4 0 0 0 0 1  | 10 0 0 0 0 
4 1 0 0 0 0  | 6 0 0 0 0 
3 2 0 0 0 0  | 0 7 0 0 0 
3 1 1 0 0 0  | 0 0 0 0 0 
3 1 0 1 0 0  | 0 0 0 0 0 
3 1 0 0 1 0  | 0 0 0 0 0 
3 1 0 0 0 1  | 0 0 0 0 0 
4 0 1 0 0 0  | 7 0 0 0 0 
3 1 1 0 0 0  | 0 0 0 0 0 
3 0 2 0 0 0  | 0 9 0 0 0 
3 0 1 1 0 0  | 0 0 0 0 0 
3 0 1 0 1 0  | 0 0 0 0 0 
3 0 1 0 0 1  | 0 0 0 0 0 
4 0 0 1 0 0  | 8 0 0 0 0 
3 1 0 1 0 0  | 0 0 0 0 0 
3 0 1 1 0 0  | 0 0 0 0 0 
3 0 0 2 0 0  | 0 11 0 0 0 
3 0 0 1 1 0  | 0 0 0 0 0 
3 0 0 1 0 1  | 0 0 0 0 0 
4 0 0 0 1 0  | 9 0 0 0 0 
3 1 0 0 1 0  | 0 0 0 0 0 
3 0 1 0 1 0  | 0 0 0 0 0 
3 0 0 1 1 0  | 0 0 0 0 0 
3 0 0 0 2 0  | 0 13 0 0 0 
3 0 0 0 1 1  | 0 0 0 0 0 
4 0 0 0 0 1  | 10 0 0 0 0 
3 1 0 0 0 1  | 0 0 0 0 0 
3 0 1 0 0 1  | 0 0 0 0 0 
3 0 0 1 0 1  | 0 0 0 0 0 
3 0 0 0 1 1  | 0 0 0 0 0 
3 0 0 0 0 2  | 0 15 0 0 0 
4 1 0 0 0 0  | 6 0 0 0 0 
3 2 0 0 0 0  | 0 7 0 0 0 
3

0 1 3 1 0 0  | 0 0 0 0 0 
0 1 2 2 0 0  | 0 0 0 0 0 
0 1 2 1 1 0  | 0 0 15 0 0 
0 1 2 1 0 1  | 0 0 0 0 0 
1 1 1 2 0 0  | 0 0 15 0 0 
0 2 1 2 0 0  | 0 0 0 0 0 
0 1 2 2 0 0  | 0 0 0 0 0 
0 1 1 3 0 0  | 0 0 0 0 0 
0 1 1 2 1 0  | 0 0 15 0 0 
0 1 1 2 0 1  | 0 0 0 0 0 
1 1 1 1 1 0  | 0 0 15 30 0 
0 2 1 1 1 0  | 0 0 15 0 0 
0 1 2 1 1 0  | 0 0 15 0 0 
0 1 1 2 1 0  | 0 0 15 0 0 
0 1 1 1 2 0  | 0 0 15 0 0 
0 1 1 1 1 1  | 0 0 15 30 0 
1 1 1 1 0 1  | 0 0 15 0 0 
0 2 1 1 0 1  | 0 0 0 0 0 
0 1 2 1 0 1  | 0 0 0 0 0 
0 1 1 2 0 1  | 0 0 0 0 0 
0 1 1 1 1 1  | 0 0 15 30 0 
0 1 1 1 0 2  | 0 0 0 0 0 
2 1 1 0 1 0  | 0 0 0 0 0 
1 2 1 0 1 0  | 0 0 0 0 0 
1 1 2 0 1 0  | 0 0 0 0 0 
1 1 1 1 1 0  | 0 0 15 30 0 
1 1 1 0 2 0  | 0 0 0 0 0 
1 1 1 0 1 1  | 0 0 0 0 0 
1 2 1 0 1 0  | 0 0 0 0 0 
0 3 1 0 1 0  | 0 0 0 0 0 
0 2 2 0 1 0  | 0 0 0 0 0 
0 2 1 1 1 0  | 0 0 15 0 0 
0 2 1 0 2 0  | 0 0 0 0 0 
0 2 1 0 1 1  | 0 0 0 0 0 
1 1 2 0 1 0  | 0 0 0 0 0 
0 2 2 0 1 0  | 0 0 0 0 0 
0 1 3 0 1 0  | 0 0 0 0 0 
0 1 2 1 1 0  | 0 0 15

1 1 1 0 2 0  | 0 0 0 0 0 
1 1 1 0 1 1  | 0 0 0 0 0 
2 0 2 0 1 0  | 0 0 0 0 0 
1 1 2 0 1 0  | 0 0 0 0 0 
1 0 3 0 1 0  | 0 0 0 0 0 
1 0 2 1 1 0  | 0 0 0 0 0 
1 0 2 0 2 0  | 0 0 0 0 0 
1 0 2 0 1 1  | 0 0 0 0 0 
2 0 1 1 1 0  | 0 0 0 0 0 
1 1 1 1 1 0  | 0 0 15 30 0 
1 0 2 1 1 0  | 0 0 0 0 0 
1 0 1 2 1 0  | 0 0 0 0 0 
1 0 1 1 2 0  | 0 0 0 0 0 
1 0 1 1 1 1  | 0 0 0 0 0 
2 0 1 0 2 0  | 0 0 0 0 0 
1 1 1 0 2 0  | 0 0 0 0 0 
1 0 2 0 2 0  | 0 0 0 0 0 
1 0 1 1 2 0  | 0 0 0 0 0 
1 0 1 0 3 0  | 0 0 0 0 0 
1 0 1 0 2 1  | 0 0 0 0 0 
2 0 1 0 1 1  | 0 0 0 0 0 
1 1 1 0 1 1  | 0 0 0 0 0 
1 0 2 0 1 1  | 0 0 0 0 0 
1 0 1 1 1 1  | 0 0 0 0 0 
1 0 1 0 2 1  | 0 0 0 0 0 
1 0 1 0 1 2  | 0 0 0 0 0 
2 1 1 0 1 0  | 0 0 0 0 0 
1 2 1 0 1 0  | 0 0 0 0 0 
1 1 2 0 1 0  | 0 0 0 0 0 
1 1 1 1 1 0  | 0 0 15 30 0 
1 1 1 0 2 0  | 0 0 0 0 0 
1 1 1 0 1 1  | 0 0 0 0 0 
1 2 1 0 1 0  | 0 0 0 0 0 
0 3 1 0 1 0  | 0 0 0 0 0 
0 2 2 0 1 0  | 0 0 0 0 0 
0 2 1 1 1 0  | 0 0 15 0 0 
0 2 1 0 2 0  | 0 0 0 0 0 
0 2 1 0 1 1  | 0 0 0 0 0 
1 1 2 0

2 0 0 2 0 1  | 0 0 0 0 0 
1 1 0 2 0 1  | 0 0 0 0 0 
1 0 1 2 0 1  | 0 0 0 0 0 
1 0 0 3 0 1  | 0 0 0 0 0 
1 0 0 2 1 1  | 0 0 0 0 0 
1 0 0 2 0 2  | 0 0 0 0 0 
1 1 0 2 0 1  | 0 0 0 0 0 
0 2 0 2 0 1  | 0 0 0 0 0 
0 1 1 2 0 1  | 0 0 0 0 0 
0 1 0 3 0 1  | 0 0 0 0 0 
0 1 0 2 1 1  | 0 0 0 0 0 
0 1 0 2 0 2  | 0 0 0 0 0 
1 0 1 2 0 1  | 0 0 0 0 0 
0 1 1 2 0 1  | 0 0 0 0 0 
0 0 2 2 0 1  | 0 0 0 0 0 
0 0 1 3 0 1  | 0 0 0 0 0 
0 0 1 2 1 1  | 0 0 15 0 0 
0 0 1 2 0 2  | 0 0 0 0 0 
1 0 0 3 0 1  | 0 0 0 0 0 
0 1 0 3 0 1  | 0 0 0 0 0 
0 0 1 3 0 1  | 0 0 0 0 0 
0 0 0 4 0 1  | 22 0 0 0 0 
0 0 0 3 1 1  | 0 0 0 0 0 
0 0 0 3 0 2  | 0 24 0 0 0 
1 0 0 2 1 1  | 0 0 0 0 0 
0 1 0 2 1 1  | 0 0 0 0 0 
0 0 1 2 1 1  | 0 0 15 0 0 
0 0 0 3 1 1  | 0 0 0 0 0 
0 0 0 2 2 1  | 0 0 0 0 0 
0 0 0 2 1 2  | 0 0 0 0 0 
1 0 0 2 0 2  | 0 0 0 0 0 
0 1 0 2 0 2  | 0 0 0 0 0 
0 0 1 2 0 2  | 0 0 0 0 0 
0 0 0 3 0 2  | 0 24 0 0 0 
0 0 0 2 1 2  | 0 0 0 0 0 
0 0 0 2 0 3  | 0 26 0 0 0 
2 0 0 1 1 1  | 0 0 0 0 0 
1 1 0 1 1 1  | 0 0 0 0 0 
1 0 1 

0 2 0 0 2 1  | 0 0 0 0 0 
0 1 1 0 2 1  | 0 0 0 0 0 
0 1 0 1 2 1  | 0 0 0 0 0 
0 1 0 0 3 1  | 0 0 0 0 0 
0 1 0 0 2 2  | 0 0 0 0 0 
1 1 0 0 1 2  | 0 0 0 0 0 
0 2 0 0 1 2  | 0 0 0 0 0 
0 1 1 0 1 2  | 0 0 0 0 0 
0 1 0 1 1 2  | 0 0 0 0 0 
0 1 0 0 2 2  | 0 0 0 0 0 
0 1 0 0 1 3  | 0 0 0 0 0 
2 1 0 0 0 2  | 0 0 0 0 0 
1 2 0 0 0 2  | 0 0 0 0 0 
1 1 1 0 0 2  | 0 0 0 0 0 
1 1 0 1 0 2  | 0 0 0 0 0 
1 1 0 0 1 2  | 0 0 0 0 0 
1 1 0 0 0 3  | 0 0 0 0 0 
1 2 0 0 0 2  | 0 0 0 0 0 
0 3 0 0 0 2  | 0 18 0 0 0 
0 2 1 0 0 2  | 0 0 0 0 0 
0 2 0 1 0 2  | 0 0 0 0 0 
0 2 0 0 1 2  | 0 0 0 0 0 
0 2 0 0 0 3  | 0 22 0 0 0 
1 1 1 0 0 2  | 0 0 0 0 0 
0 2 1 0 0 2  | 0 0 0 0 0 
0 1 2 0 0 2  | 0 0 0 0 0 
0 1 1 1 0 2  | 0 0 0 0 0 
0 1 1 0 1 2  | 0 0 0 0 0 
0 1 1 0 0 3  | 0 0 0 0 0 
1 1 0 1 0 2  | 0 0 0 0 0 
0 2 0 1 0 2  | 0 0 0 0 0 
0 1 1 1 0 2  | 0 0 0 0 0 
0 1 0 2 0 2  | 0 0 0 0 0 
0 1 0 1 1 2  | 0 0 0 0 0 
0 1 0 1 0 3  | 0 0 0 0 0 
1 1 0 0 1 2  | 0 0 0 0 0 
0 2 0 0 1 2  | 0 0 0 0 0 
0 1 1 0 1 2  | 0 0 0 0 0 
0 1 0 1 1 

In [5]:
def train(net, train_iter,criterion, num_epochs, device,lr=None, weight_decay = 0):

    print('training on', device)
    net.to(device)
    #optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay = weight_decay)
    optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay = weight_decay)
    
    for epoch in range(num_epochs):
        #train_l_sum = torch.tensor([0.0], dtype=torch.float32, device=device)
        #train_loss_sum = torch.tensor([0.0], dtype=torch.float32, device=device)
        train_l_sum = 0.0
        train_loss_sum = 0.0
        
        
        n, start = 0, time.time()

        for x, y in train_iter:
            net.train()
            optimizer.zero_grad()
            x,y = x.to(device),y.to(device)
            
            y_hat = net(x)
            loss = criterion(y, y_hat)
            #loss = criterion(torch.atanh(y), torch.atanh(y_hat))
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                train_l_sum += loss.float()
                train_loss_sum += sum([abs(unpack(d)) for d in (y-y_hat)])
                n += 1


        print('epoch %d, loss %.4f, train loss %.3f, time %.1f sec, n=%d' %
              (epoch + 1, train_l_sum/n*SCALE, train_loss_sum/n, time.time() - start, n) )
    return

In [15]:
criterion = nn.MSELoss(reduction = 'sum')
train(net, train_data, criterion,100, device, 0.000001)

training on cuda:0
epoch 1, loss 0.0598, train loss 0.188, time 9.9 sec, n=7776
epoch 2, loss 0.0598, train loss 0.188, time 9.5 sec, n=7776


KeyboardInterrupt: 

In [7]:
torch.save(net.state_dict(), './data/HM_learn_scores_' + str(1))

In [20]:
corr = 0

for x,y in train_data:
    net.eval()
    x,y = x.to(device),y.to(device)
    y_hat = net(x)

    hy = y.clone().detach()
    hyh = y_hat.clone().detach()
    
    dh = (hy-hyh)
    
    flag = False

    
    for i in range(OUTPUT_SIZE):
        if not unpack(dh[i]) == 0:
            flag = True
            break
    
    if not flag:
        corr += 1
        continue
    
    cont = ""
    for a in x:
        cont += str(int(a)) + " "
    cont += " | "
    for d in hy-hyh:
    #for d in hyh:
        cont += str(unpack(d)) + " "
    print(cont)
        
print("{0}".format(corr))


5 0 0 0 0 0  | 3 4 0 0 50 
4 1 0 0 0 0  | 6 -1 0 0 0 
4 0 1 0 0 0  | 6 -1 0 0 0 
4 0 0 1 0 0  | 8 -1 0 0 0 
4 0 0 0 1 0  | 9 -1 0 0 0 
4 0 0 0 0 1  | 10 0 0 0 0 
4 1 0 0 0 0  | 6 -1 0 0 0 
3 2 0 0 0 0  | 0 6 0 0 0 
4 0 1 0 0 0  | 6 -1 0 0 0 
3 0 2 0 0 0  | 0 8 0 0 0 
3 0 1 0 1 0  | 0 -1 0 0 0 
4 0 0 1 0 0  | 8 -1 0 0 0 
4 0 0 0 1 0  | 9 -1 0 0 0 
3 0 1 0 1 0  | 0 -1 0 0 0 
4 0 0 0 0 1  | 10 0 0 0 0 
3 0 0 0 0 2  | 0 0 0 0 -1 
4 1 0 0 0 0  | 6 -1 0 0 0 
3 2 0 0 0 0  | 0 6 0 0 0 
3 2 0 0 0 0  | 0 6 0 0 0 
2 3 0 0 0 0  | 0 7 0 0 0 
4 0 1 0 0 0  | 6 -1 0 0 0 
3 0 2 0 0 0  | 0 8 0 0 0 
3 0 1 0 1 0  | 0 -1 0 0 0 
3 0 2 0 0 0  | 0 8 0 0 0 
3 0 1 0 1 0  | 0 -1 0 0 0 
4 0 0 1 0 0  | 8 -1 0 0 0 
4 0 0 0 1 0  | 9 -1 0 0 0 
3 0 1 0 1 0  | 0 -1 0 0 0 
3 0 1 0 1 0  | 0 -1 0 0 0 
4 0 0 0 0 1  | 10 0 0 0 0 
3 0 0 0 0 2  | 0 0 0 0 -1 
3 0 0 0 0 2  | 0 0 0 0 -1 
2 0 0 0 0 3  | 1 0 0 0 -2 
4 1 0 0 0 0  | 6 -1 0 0 0 
3 2 0 0 0 0  | 0 6 0 0 0 
3 2 0 0 0 0  | 0 6 0 0 0 
2 3 0 0 0 0  | 0 7 0 0 0 
3 2 0 0 0 0

1 0 0 0 0 4  | 1 0 0 0 -6 
0 4 0 0 0 1  | 1 0 0 0 0 
0 1 1 0 0 3  | -1 0 0 0 0 
0 1 0 0 1 3  | 0 -1 0 0 0 
0 1 1 0 0 3  | -1 0 0 0 0 
0 1 0 0 1 3  | 0 -1 0 0 0 
0 1 1 0 0 3  | -1 0 0 0 0 
0 1 0 0 1 3  | 0 -1 0 0 0 
0 1 0 0 0 4  | 1 0 0 0 -4 
0 1 1 0 0 3  | -1 0 0 0 0 
0 0 4 0 0 1  | 2 -1 0 0 -4 
0 0 3 0 0 2  | 0 0 0 0 -1 
0 0 3 0 0 2  | 0 0 0 0 -1 
0 0 2 0 0 3  | 0 0 0 0 -1 
0 0 1 3 0 1  | 0 0 0 0 1 
0 0 1 2 0 2  | 0 0 0 0 1 
0 0 1 2 0 2  | 0 0 0 0 1 
0 1 1 0 0 3  | -1 0 0 0 0 
0 0 3 0 0 2  | 0 0 0 0 -1 
0 0 2 0 0 3  | 0 0 0 0 -1 
0 0 1 2 0 2  | 0 0 0 0 1 
0 1 1 0 0 3  | -1 0 0 0 0 
0 0 2 0 0 3  | 0 0 0 0 -1 
0 0 1 0 0 4  | 2 0 0 0 -5 
0 0 1 3 0 1  | 0 0 0 0 1 
0 0 1 2 0 2  | 0 0 0 0 1 
0 0 1 2 0 2  | 0 0 0 0 1 
0 0 1 3 0 1  | 0 0 0 0 1 
0 0 1 2 0 2  | 0 0 0 0 1 
0 0 1 3 0 1  | 0 0 0 0 1 
0 0 0 4 0 1  | 0 1 0 0 -4 
0 0 1 2 0 2  | 0 0 0 0 1 
0 0 0 2 0 3  | 0 0 0 0 -1 
0 0 1 2 0 2  | 0 0 0 0 1 
0 0 1 2 0 2  | 0 0 0 0 1 
0 0 0 2 0 3  | 0 0 0 0 -1 
0 0 0 2 0 3  | 0 0 0 0 -1 
0 0 0 1 0 4  |

t1=torch.tensor([1,1,1,1],dtype = torch.float32,requires_grad = False)
t2=torch.tensor([1,2,3,4],dtype = torch.float32,requires_grad = True)

optimizer = optim.Adam([t2],lr=1)
for i in range(1000):
    tst = nn.L1Loss(reduction = 'mean')(t1,t2)
    
    optimizer.zero_grad()
    tst.backward()
    optimizer.step()
    
print(str(float(tst)))
print(str(t2.tolist()))

In [12]:
print(net(torch.tensor([0,0,0,0,1,4],dtype=torch.float32, device=device)))

tensor([ 0.8701,  0.0036, -0.0023, -0.0017,  0.1917], device='cuda:0',
       grad_fn=<AddBackward0>)


In [18]:
print(net(torch.tensor([0,0,0,0,0,5],dtype=torch.float32, device=device)))

tensor([1.4798e+00, 9.7084e-01, 9.2735e-03, 1.3087e-03, 3.5243e-01],
       device='cuda:0', grad_fn=<AddBackward0>)


In [17]:
print(net(torch.tensor([1,1,1,1,1,0],dtype=torch.float32, device=device)))

tensor([ 1.0458e-03,  2.0774e-03,  4.9961e-01,  9.9933e-01, -7.0084e-05],
       device='cuda:0', grad_fn=<AddBackward0>)
